# Working with Custom Trace Formats

This notebook shows how to use Toolscore with custom trace formats beyond OpenAI and Anthropic.

## What You'll Learn

1. Understanding the custom format structure
2. Creating custom gold standards
3. Evaluating custom traces
4. Best practices for custom formats

In [ ]:
import sys
sys.path.insert(0, '../..')  # For development

# Load environment variables (optional, for LLM judge metrics)
from pathlib import Path
env_path = Path('../../.env')
if env_path.exists():
    from dotenv import load_dotenv
    load_dotenv(env_path)
    print("✓ Environment variables loaded")

from toolscore import evaluate_trace
import json
from pathlib import Path

## 1. Custom Format Structure

The custom format is flexible and can handle various structures:

In [ ]:
# Example 1: Dictionary with 'calls' array
custom_format_1 = {
    "calls": [
        {
            "tool": "search_web",
            "args": {"query": "Python tutorials", "num_results": 10},
            "result": "Found 10 results"
        },
        {
            "tool": "summarize",
            "args": {"text": "..."},
            "result": "Summary created"
        }
    ]
}

print("Custom Format 1:")
print(json.dumps(custom_format_1, indent=2))

In [ ]:
# Example 2: Direct array of calls
custom_format_2 = [
    {
        "name": "read_file",  # 'name' or 'tool' both work
        "arguments": {"path": "data.txt"},  # 'arguments' or 'args' both work
    },
    {
        "tool": "write_file",
        "args": {"path": "output.txt", "content": "processed data"},
    }
]

print("Custom Format 2:")
print(json.dumps(custom_format_2, indent=2))

## 2. Create a Custom Trace

Let's create a complete custom trace for evaluation:

In [ ]:
# Custom trace for a file processing workflow
custom_trace = {
    "calls": [
        {
            "tool": "read_file",
            "args": {"filename": "input.txt"},
            "result": "File content loaded",
            "timestamp": 1234567890.0,
            "duration": 0.05
        },
        {
            "tool": "process_text",
            "args": {"text": "...", "operation": "uppercase"},
            "result": "Processed successfully",
            "duration": 0.12
        },
        {
            "tool": "write_file",
            "args": {"filename": "output.txt", "content": "..."},
            "result": "File written",
            "duration": 0.03
        }
    ]
}

# Save to file
with open("custom_trace.json", "w") as f:
    json.dump(custom_trace, f, indent=2)

print("✅ Custom trace created")

## 3. Create Gold Standard

Define the expected behavior:

In [ ]:
gold_standard = [
    {
        "tool": "read_file",
        "args": {"filename": "input.txt"},
        "description": "Read input file",
        "side_effects": {
            "file_exists": "input.txt"
        }
    },
    {
        "tool": "process_text",
        "args": {"operation": "uppercase"},
        "description": "Process text to uppercase"
    },
    {
        "tool": "write_file",
        "args": {"filename": "output.txt"},
        "description": "Write processed output",
        "side_effects": {
            "file_exists": "output.txt"
        }
    }
]

# Save gold standard
with open("custom_gold.json", "w") as f:
    json.dump(gold_standard, f, indent=2)

print("✅ Gold standard created")

## 4. Run Evaluation

Evaluate the custom trace:

In [ ]:
result = evaluate_trace(
    gold_file="custom_gold.json",
    trace_file="custom_trace.json",
    format="custom"  # Explicitly specify custom format
)

print("=== Evaluation Results ===")
print(f"Invocation Accuracy: {result.metrics['invocation_accuracy']:.1%}")
print(f"Selection Accuracy:  {result.metrics['selection_accuracy']:.1%}")
print(f"Sequence Accuracy:   {result.metrics['sequence_metrics']['sequence_accuracy']:.1%}")
print(f"Argument F1:         {result.metrics['argument_metrics']['f1']:.1%}")

# Output example:
# === Evaluation Results ===
# Invocation Accuracy: 100.0%
# Selection Accuracy:  100.0%
# Sequence Accuracy:   100.0%
# Argument F1:         66.7%  (some args may differ)

## 5. Performance Metrics

Custom traces can include performance data:

In [ ]:
# Check if latency metrics are available
if 'latency_metrics' in result.metrics:
    lat = result.metrics['latency_metrics']
    print("Performance Metrics:")
    print(f"Total Duration:   {lat['total_duration']:.3f}s")
    print(f"Average Duration: {lat['average_duration']:.3f}s")
    print(f"Max Duration:     {lat['max_duration']:.3f}s")
    print(f"Min Duration:     {lat['min_duration']:.3f}s")
else:
    print("No latency data in trace")

## 6. Auto-Detection

Toolscore can auto-detect custom formats:

In [ ]:
# Use auto-detection
result_auto = evaluate_trace(
    gold_file="custom_gold.json",
    trace_file="custom_trace.json",
    format="auto"  # Will detect it's a custom format
)

print(f"Auto-detected format evaluation:")
print(f"Selection Accuracy: {result_auto.metrics['selection_accuracy']:.1%}")

## 7. Best Practices

### Tips for Custom Formats:

1. **Use consistent field names**: Stick to either `tool`/`args` or `name`/`arguments`
2. **Include metadata**: Add timestamps, durations, costs when available
3. **Document your format**: Keep a schema for your custom format
4. **Test incrementally**: Start with simple cases before complex workflows
5. **Use side-effects**: Validate important outcomes (files created, API calls, etc.)

## 8. Real-World Example

Here's a more complex custom trace:

In [ ]:
complex_trace = {
    "metadata": {
        "agent_id": "agent-123",
        "session_id": "session-456",
        "timestamp": "2025-10-13T12:00:00Z"
    },
    "calls": [
        {
            "tool": "api_call",
            "args": {
                "url": "https://api.example.com/users",
                "method": "GET"
            },
            "result": {"status": 200, "users": [...]},
            "duration": 0.5,
            "cost": 0.0001
        },
        {
            "tool": "filter_data",
            "args": {"condition": "active_users"},
            "result": {"count": 42},
            "duration": 0.02
        },
        {
            "tool": "generate_report",
            "args": {"format": "pdf", "template": "summary"},
            "result": {"file": "report.pdf", "pages": 5},
            "duration": 1.2
        }
    ]
}

print("Complex custom trace structure:")
print(json.dumps(complex_trace, indent=2)[:500], "...")

## Summary

In this notebook, you learned:

✅ How to structure custom trace formats

✅ How to create gold standards for custom workflows

✅ How to evaluate custom traces

✅ How to include performance metrics

✅ Best practices for custom format design

## Next Steps

- Check out `03_advanced_metrics.ipynb` for deep dives into specific metrics
- Adapt this approach to your own agent framework
- Read the [custom adapter documentation](https://toolscore.readthedocs.io/en/latest/api/adapters.html)